In [6]:
from pyhive import presto
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
import numpy as np
import time

from datetime import datetime, timedelta

import seaborn as sns

conn = presto.connect(
    host='presto.processing.yoda.run', 
    ## presto.processing.yoda.run
    ## bi-presto.serving.data.production.internal
    port=80,
    username='manoj.ravirajan@rapido.bike'
)

In [7]:
start_date = '20230313'
start_date1 = '2023-03-13'
end_date = '20230319'
end_date1 = '2023-03-19'

In [8]:
query = f"""

with base as (
select
    date_format(DATE_TRUNC('week', date_parse(yyyymmdd, '%Y%m%d')), '%Y-%m-%d') order_week,
    coalesce(pickup_cluster, 'NO_Cluster') pickup_cluster,
    customer_id,
    order_id
    
from orders.order_logs_snapshot 
where 
    yyyymmdd BETWEEN '{start_date}'
    and '{end_date}'
    and order_status = 'dropped'
    and city_name = 'Hyderabad' 
    and service_obj_service_name in ('Auto', 'Link')
    and (spd_fraud_flag != true OR spd_fraud_flag IS NULL)
),

segment as (
select 
    date_format(DATE_TRUNC('week', cast(run_date as date)), '%Y-%m-%d') week,
    customer_id,
    taxi_income_segment,
    ps_tag_auto,
    ps_tag_link
    
from datasets.iallocator_customer_segments
where 
    run_date BETWEEN '{start_date1}'
    and '{end_date1}'
    and day_of_week(cast (run_date as date)) = 7
    and customer_id in (select distinct customer_id from base)
),

cluster_level as (
select 
    order_week,
    pickup_cluster,
    count(distinct order_id) cluster_orders,
    count(distinct customer_id) cluster_net_customers
from base
group by 1,2
),

detailed as (
select 
    base.order_week,
    pickup_cluster,
    case 
    when taxi_income_segment in ('MEDIUM_INCOME', 'LOW_INCOME') then 'LOW_INCOME'
    else taxi_income_segment end as income_segment,
    ps_tag_link,
    ps_tag_auto,
    
    count(distinct base.order_id) total_orders,
    count(distinct base.customer_id) net_customers
from base
left join segment on base.order_week = segment.week and base.customer_id = segment.customer_id
group by 1,2,3,4,5
)


select 
    c.*,
    d.income_segment,
    d.ps_tag_link,
    d.ps_tag_auto,
    d.total_orders segment_level_orders,
    try(d.total_orders*100.00/c.cluster_orders) "segment_level_orders%",
    d.net_customers segment_level_customers,
    try(d.net_customers*100.00/c.cluster_net_customers) "segment_level_customers%"
from cluster_level c
left join detailed d on c.order_week = d.order_week and c.pickup_cluster = d.pickup_cluster 
order by 3 desc,4 desc, 5, 6, 7

"""

In [9]:
df_code = pd.read_sql(query, conn)
df_code 

KeyboardInterrupt: 

In [ ]:
df_code

NameError: name 'df_code' is not defined

In [6]:
df_code.to_csv("hyd-cluster.csv", index = False)